In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn
import os
import os.path as osp
import collections
import PIL
import imageio
from distutils.version import LooseVersion
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


In [2]:
# Downloading dataset from google drive, 밑의 코드의 주석을 풀면 구글 드라이브로 부터 데이터셋 다운, 압축이 풀리고, Kitti라는 폴더가 생성됩니다. 

# !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=19EiycfOQtf6uDKvMgwlHZB50cAxX_U4z" -O Kitti.zip && rm -rf /tmp/cookies.txt
# !mkdir Kitti
# !unzip Kitti.zip -d Kitti


# Dataloader

In [3]:
imgsets_file = osp.join('Kitti', '{}.txt'.format('train'))
for line in open(imgsets_file):
    line = line.strip()
    print(line)
    line = line.split()

training/image_2/um_000000.png training/gt_image_2/um_road_000000.png
training/image_2/um_000001.png training/gt_image_2/um_road_000001.png
training/image_2/um_000002.png training/gt_image_2/um_road_000002.png
training/image_2/um_000003.png training/gt_image_2/um_road_000003.png
training/image_2/um_000004.png training/gt_image_2/um_road_000004.png
training/image_2/um_000005.png training/gt_image_2/um_road_000005.png
training/image_2/um_000006.png training/gt_image_2/um_road_000006.png
training/image_2/um_000007.png training/gt_image_2/um_road_000007.png
training/image_2/um_000008.png training/gt_image_2/um_road_000008.png
training/image_2/um_000009.png training/gt_image_2/um_road_000009.png
training/image_2/um_000010.png training/gt_image_2/um_road_000010.png
training/image_2/um_000011.png training/gt_image_2/um_road_000011.png
training/image_2/um_000012.png training/gt_image_2/um_road_000012.png
training/image_2/um_000013.png training/gt_image_2/um_road_000013.png
training/image_2/um_

In [4]:
class KITTIdataset(torch.utils.data.Dataset):
    class_names = np.array(['background', 'road'])

    def __init__(self, root, transform, split='train'): # root: "./Kitti"
        self.root = root
        self.split = split
        self.transform = transform

        self.image_path = []
        self.ys = []
        
        imgsets_file = osp.join(root, '{}.txt'.format(split))
        for did in open(imgsets_file):
            did = did.strip()
            did = did.split()
            img_file = osp.join(root, 'data_road/{}'.format(did[0]))
            lbl_file = osp.join(root, 'data_road/{}'.format(did[1]))
            self.image_path.append(img_file)
            self.ys.append(lbl_file)

    def __len__(self):
        return len(self.ys)

    def __getitem__(self, index):
        # load image
        img_file = self.image_path[index]
        img = PIL.Image.open(img_file)
        img = np.array(img)
        
        # load label
        lbl_file = self.ys[index]
        lbl = PIL.Image.open(lbl_file)
        lbl = np.array(lbl)
        lbl[lbl == 255] = 1 # 0 is black 255 is white
        
        return self.transform(img), torch.from_numpy(lbl).long()

## Define dataset and dataloader

In [5]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])
train_dataset = KITTIdataset(root = './Kitti', split = 'train', transform = transform)
val_dataset = KITTIdataset(root = './Kitti', split = 'val', transform = transform)

print(train_dataset[0])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = 1, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = 1, shuffle = False)

(tensor([[[ 1.0000,  1.0000,  1.0000,  ..., -0.1216, -0.2392, -0.2863],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.8353, -0.8196, -0.8039],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.7098, -0.6157, -0.4353],
         ...,
         [-0.4510, -0.4431, -0.3804,  ..., -0.2549, -0.2549, -0.2471],
         [-0.4196, -0.4275, -0.3725,  ..., -0.2549, -0.2627, -0.2706],
         [-0.3882, -0.3882, -0.4745,  ..., -0.2235, -0.2392, -0.2549]],

        [[ 1.0000,  1.0000,  1.0000,  ...,  0.0431,  0.0431, -0.1216],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.4039, -0.4980, -0.4824],
         [ 1.0000,  1.0000,  1.0000,  ..., -0.7725, -0.5922, -0.3020],
         ...,
         [-0.3490, -0.3569, -0.3725,  ..., -0.2235, -0.2471, -0.2706],
         [-0.4196, -0.4431, -0.4510,  ..., -0.2471, -0.2392, -0.2549],
         [-0.4510, -0.4353, -0.4118,  ..., -0.2627, -0.2627, -0.2627]],

        [[ 1.0000,  1.0000,  1.0000,  ..., -0.6235, -0.5843, -0.5765],
         [ 1.0000,  1.0000,  1.0000,  ..., -

# Define the Network
-VGG16

- FCN model

In [6]:
class FCN(nn.Module):
    def __init__(self, num_class = 3):
        super(FCN, self).__init__()
        
        ## Why padding 100?? https://github.com/shelhamer/fcn.berkeleyvision.org
        self.features1 = nn.Sequential(
            nn.Conv2d(3, 64, 3, padding = 100),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding = 1),
            nn.ReLU())
    
        self.features2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding = 1),
            nn.ReLU())
#         self.pool2 = nn.MaxPool2d(2, stride = 2, ceil_mode = True)    
        
        self.features3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding = 1))
#         self.pool3 = nn.MaxPool2d(2, stride = 2, ceil_mode = True)        
        
        self.features4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1))
#         self.pool4 = nn.MaxPool2d(2, stride = 2, ceil_mode = True)
                
        self.features5 = nn.Sequential(
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1),
            nn.ReLU(),
            nn.Conv2d(512, 512, 3, padding = 1))
#         self.pool5 = nn.MaxPool2d(2, stride = 2, ceil_mode = True)
        
        self.maxpool = nn.MaxPool2d(2, stride = 2, ceil_mode = True)

        self.classifier = nn.Sequential(
            nn.Conv2d(512, 4096, 7),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Conv2d(4096, 4096, 1),
            nn.ReLU(),
            nn.Dropout2d(),
            nn.Conv2d(4096, num_class, 1))
        
        self.upscore2 = nn.ConvTranspose2d(num_class, num_class, kernel_size = 4, stride = 2, bias = False)
        self.upscore4 = nn.ConvTranspose2d(num_class, num_class, kernel_size = 4, stride = 2, bias = False)
        self.upscore8 = nn.ConvTranspose2d(num_class, num_class, kernel_size = 16, stride = 8, bias = False)
        
        self.score_pool4 = nn.Conv2d(512, num_class, 1)
        self.score_pool3 = nn.Conv2d(256, num_class, 1)
        
        self.params = [self.features1, self.features2, self.features3, 
                       self.features4, self.features5]
        

                             
    def forward(self, inputs):
        # input [Batch size, 3, w, h]
        x = self.features1(inputs) # [Batch size, 64, w, h]
        pool1 = self.maxpool(x) # [Batch size, 64, w/2, h/2]
        
        x = self.features2(pool1) # [Batch size, 128, w/2, h/2]
        pool2 = self.maxpool(x) # [Batch size, 128, w/4, h/4]
        
        x = self.features3(pool2) # [Batch size, 256, w/4, h/4]
        pool3 = self.maxpool(x) # [Batch size, 256, w/8, h/8]
        
        x = self.features4(pool3) # [Batch size, 512, w/8, h/8]
        pool4 = self.maxpool(x) # [Batch size, 512, w/16, h/16]
        
        x = self.features5(pool4) # [Batch size, 512, w/16, h/16]
        pool5 = self.maxpool(x) # [Batch size, 512, w/32, h/32]
        
        x = self.classifier(pool5) # [w/32 - 6, h/32 - 6]
        
        x = self.upscore2(x) # [w/16 - 10, h/16 - 10]
        
        pool4 = self.score_pool4(pool4)
        pool4 = pool4[:, :, 5:5 + x.size()[2], 5:5 + x.size()[3]]
        x = torch.add(x, pool4) # add together 
        
        x = self.upscore4(x) # [w/8 - 18, h/8 - 18]
        
        pool3 = self.score_pool3(pool3)
        pool3 = pool3[:, :, 9:9 + x.size()[2], 9:9 + x.size()[3]]
        x = torch.add(x, pool3) # add together

        x = self.upscore8(x) 
        x = x[:, :, 33:33 + inputs.size()[2], 33:33 + inputs.size()[3]]
        
        return x
    
    def copy_params(self, vgg):
        for l1, l2 in zip(vgg.features, self.params):
            if (isinstance(l1, nn.Conv2d) and isinstance(l2, nn.Conv2d)):
                assert l1.weight.size() == l2.weight.size()
                assert l1.bias.size() == l2.bias.size()
                l2.weight.data = l1.weight.data
                l2.bias.data = l1.bias.data

In [7]:
import torch.nn as nn
import torch

test = torch.randn(2,3,32,32)
x = torch.tensor([[-2, 1, 2, 6, 4], [-3, 1, 7, 2, -2], [-4, 2, 3, -1 , -3], [-7, 1, 2, 3, 11], [5, -7, 8, 12, -9]]).float()
x = x.unsqueeze(0)
y_1 = nn.MaxPool2d(kernel_size=2,stride=2, padding=0)
y_2 = nn.MaxPool2d(kernel_size=2,stride=2, padding=0, ceil_mode=True)
print(x)
print(y_1(x))
print(y_2(x))

tensor([[[-2.,  1.,  2.,  6.,  4.],
         [-3.,  1.,  7.,  2., -2.],
         [-4.,  2.,  3., -1., -3.],
         [-7.,  1.,  2.,  3., 11.],
         [ 5., -7.,  8., 12., -9.]]])
tensor([[[1., 7.],
         [2., 3.]]])
tensor([[[ 1.,  7.,  4.],
         [ 2.,  3., 11.],
         [ 5., 12., -9.]]])


In [8]:
m1 = nn.ConvTranspose2d(2, 2, kernel_size = 4, stride = 2, bias = False)
m2 = nn.ConvTranspose2d(2, 2, kernel_size = 4, stride = 2, bias = False)
m3 = nn.ConvTranspose2d(2, 2, kernel_size = 16, stride = 8, bias = False)
temp_input = torch.randn(1,2,50,50)
temp_output = m3(temp_input)
print(temp_input.size())
print(temp_output.size())

torch.Size([1, 2, 50, 50])
torch.Size([1, 2, 408, 408])


In [9]:
# model = FCN(3)
# # print(model)
temp_input = torch.rand(1, 3, 1024, 1024)
# output = model(temp_input)

## Measure accuracy and visualization

In [31]:
def _fast_hist(label_true, label_pred, n_class):
    mask = (label_true >= 0) & (label_true < n_class)
    hist = np.bincount(
        n_class * label_true[mask].astype(int) +
        label_pred[mask], minlength=n_class**2).reshape(n_class, n_class)
    return hist

def compute_mean_iou(label_trues, label_preds, n_class):
    hist = np.zeros((n_class, n_class))
    for lt, lp in zip(label_trues, label_preds):
        hist += _fast_hist(lt.flatten(), lp.flatten(), n_class)
    iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))
    mean_iou = np.nanmean(iu)
    
    return mean_iou

def visualization(net, input_img, epoch):
    img = transform(input_img).unsqueeze(0)
    img = img.cuda()
    
    score = net(img)
#     score = score["out"]
    
    os.makedirs("./pred", exist_ok = True)
    os.makedirs("./input", exist_ok = True)
    _, lbl_pred = score.max(1)
    lbl_pred = lbl_pred.cpu().numpy()  
    lbl_pred = np.squeeze(lbl_pred)
    imageio.imsave('./pred/mask_pretrained_'+str(epoch+1)+'.png', lbl_pred)
    plt.imshow(mpimg.imread('./pred/mask_pretrained_'+str(epoch+1)+'.png')) ### visualize predicted label map

    input_img = np.array(input_img)
    imageio.imsave('./input/input_pretrained_'+str(epoch+1)+'.png', input_img)
    plt.imshow(mpimg.imread('./input/input_pretrained_'+str(epoch+1)+'.png'))
    

In [33]:
label_true = np.array([
    [1, 2], 
    [2, 2]
])
label_pred = np.array([
    [1, 2], 
    [2, 3]
])
compute_mean_iou(label_true, label_pred, 4)

[1 2]
[2 2]
[[0. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 2. 1.]
 [0. 0. 0. 0.]]
[       nan 1.         0.66666667 0.        ]


<ipython-input-31-06631ce91ffc>:13: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


0.5555555555555555

# Train

In [11]:
# ##load pretrained model from torchvision
# ##pretrained using coco 2017
# net = torchvision.models.segmentation.fcn_resnet50(pretrained = True)
# # print(net)
# net.classifier = torchvision.models.segmentation.fcn.FCNHead( 2048, 2)
# net = net.cuda()
# print(net)

FCN(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequenti

In [12]:
# vgg16 = torchvision.models.vgg16(pretrained = True)
# net = FCN(num_class = 2)
# net.copy_params(vgg16)

# net = net.cuda()

In [13]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss() 
optimizer = optim.Adam(net.parameters(), lr=1e-5, weight_decay = 0.0001)

In [21]:
training_epochs = 5
best_iou = 0
num_class = len(train_loader.dataset.class_names)
j=0

for epoch in range(training_epochs):
    net.train()
    print ('current epoch : %d'%(epoch))
    running_loss = 0.0

    for batch_idx, (data, target) in enumerate(train_loader):
        # load data, forward
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        
        score = net(data)
#         import pdb; pdb.set_trace()
#         score = score["out"]

        loss = criterion(score, target)
        loss.backward()
        optimizer.step()
        
        if batch_idx % 20 ==0:
            print ('batch : {}, loss : {}'.format(batch_idx, loss.item()))
        j += 1
        
    #validation
    net.eval()
    val_loss = 0
    metrics = []
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            # load data, forward
            data, target = data.cuda(), target.cuda()
            score = net(data)
            ## check score output
            score = score["out"]

            # calc val loss, accuracy
            loss = criterion(score, target)

            val_loss += loss.item()

            _, lbl_pred = score.max(1)
            lbl_pred = lbl_pred.cpu().numpy()  
            lbl_true = target.cpu().numpy()

            for lt, lp in zip(lbl_true, lbl_pred): # lbl_true, lbl_pred: [batch, h, w]
                tmp = compute_mean_iou(lt, lp, num_class)
                metrics.append(tmp)
            
    val_loss /= len(val_loader)
    metrics = np.mean(metrics)
    
    print ('val loss : {}, mean_iou : {}'.format(val_loss, metrics))

    ##save model
    ## code 실수
    if best_iou < metrics:
        best_iou = metrics
        print("Best model saved")
        torch.save(net.state_dict(), './model_best.pth')
    
    ## visualization
    img = PIL.Image.open('./road_sample1.png')
    visualization(net, img, epoch)
print('Finished Training')

current epoch : 0
Best model saved


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 10.91 GiB total capacity; 4.90 GiB already allocated; 5.38 MiB free; 5.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
img = PIL.Image.open('./road_sample1.png')
visualization(net, img, epoch)